Before starting, please run the following code in a terminal (in a new tab)

```
module purge
module load ollama
export HUGGINGFACE_HUB_CACHE=/pbs/throng/training/universite-hiver/cache/huggingface
export OLLAMA_MODELS=/pbs/throng/training/universite-hiver/cache/ollama/models
export OLLAMA_HOST=127.0.0.1:65383
ollama serve &
# ollama pull gpt-oss:20b
```

This will let run an Ollama backend to run an LLM (GPT-oss-20b) for the "Generation" part in RAG.

# A Starting Example

Retrieval-Augmented Generation (RAG) is a technique that enhances the output of language models by grounding their responses in external, relevant documents. Instead of relying solely on model parameters, RAG systems first retrieve context passages related to a user query and then generate an answer based on that evidence. This improves factual accuracy, transparency, and adaptability to custom datasets.

In this example, we implement a simple RAG pipeline using U.S. presidential speeches as our knowledge base. We'll walk through the following steps:

- **Data Preparation**: Load and explore a dataset of presidential speech transcripts.
- **TF-IDF Retrieval**: Convert speeches into vector representations using TF-IDF (keywords) and retrieve the most relevant ones based on cosine similarity.
- **Query Answering with Ollama**: Use a locally hosted language model (via Ollama) to generate a natural language answer using the retrieved speech excerpts as context.
- **Keyword Match Exploration**: Inspect which terms from the query appear in each retrieved document to better understand the retrieval step.
- **Embedding-based Retrieval**: Retrieve sematically relevant documents intead of keywords.

This setup demonstrates a lightweight but effective RAG workflow suitable for text collections in the social sciences, especially where domain-specific knowledge is locked inside qualitative documents such as interviews, policy statements, or historical texts.

## Import Dependencies

In [1]:
import numpy as np
import pandas as pd

from ollama import Client

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

## Load data

In [2]:
data1 = pd.read_csv('data/speeches.csv')

In [3]:
data1.head()

,title,date,transcript,president
0,"July 2, 1807: Proclamation in Response to the ...",1807-07-02T13:03:58-04:56,During the wars which for some time have unhap...,Thomas Jefferson
1,"September 26, 1963: Address at the Mormon Tabe...",1963-09-26T13:00:00-04:00,"Senator Moss, my old colleague in the United S...",John F. Kennedy
2,"October 26, 2020: Swearing-In Ceremony of the ...",2020-10-26T21:00:00-04:00,THE PRESIDENT: Thank you very much. Appreciate...,Donald Trump
3,"December 3, 1900: Fourth Annual Message",1900-12-03T13:00:00-05:00,To the Senate and House of Representatives: \n...,William McKinley
4,"December 2, 1806: Sixth Annual Message",1806-12-02T13:03:58-04:56,TO THE SENATE AND HOUSE OF REPRESENTATIVES OF ...,Thomas Jefferson


In [4]:
len(data1)

1054

In [5]:
print(data1['transcript'].values[0])

During the wars which for some time have unhappily prevailed among the powers of Europe the United States of America, firm in their principles of peace, have endeavored, by justice, by a regular discharge of all their national and social duties, and by every friendly office their situation has admitted, to maintain with all the belligerents their accustomed relations of friendship, hospitality, and commercial intercourse. Taking no part in the questions which animate these powers against each other, nor permitting themselves to entertain a wish but for the restoration of general peace, they have observed with good faith the neutrality they assumed, and they believe that no instance of a departure from its duties can be justly imputed to them by any nation. A free use of their harbors and waters, the means of refitting and of refreshment, of succor to their sick and suffering, have at all times and on equal principles been extended to all, and this, too, amidst a constant recurrence of 

In [6]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(data1['transcript'])

In [7]:
tfidf_matrix.shape

(1054, 1000)

In [8]:
vectorizer.get_feature_names_out()

array(['000', '10', '100', '11', '12', '15', '20', '25', '30', '50',
       '500', 'ability', 'able', 'abroad', 'accept', 'accomplished',
       'according', 'account', 'achieve', 'actions', 'acts', 'actual',
       'actually', 'add', 'added', 'addition', 'additional', 'address',
       'adequate', 'adjustment', 'administration', 'adopted', 'advance',
       'advantage', 'affairs', 'afford', 'afghanistan', 'africa', 'age',
       'agents', 'aggression', 'agree', 'agreed', 'agreement',
       'agricultural', 'agriculture', 'ahead', 'aid', 'air', 'allies',
       'allow', 'allowed', 'amendment', 'americans', 'annual', 'answer',
       'applause', 'application', 'applied', 'appointed', 'appropriate',
       'appropriation', 'appropriations', 'approval', 'approved', 'april',
       'area', 'areas', 'armed', 'arms', 'army', 'article', 'articles',
       'asia', 'ask', 'asked', 'assistance', 'atlantic', 'attack',
       'attempt', 'attention', 'august', 'authorities', 'authority',
       'au

### helper functions

In [9]:
def retrieve_relevant_speeches(data, tfidf_matrix, query, top_k=5):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = np.argsort(similarity)[-top_k:][::-1]
    return data.iloc[top_indices], similarity[top_indices]

In [10]:
def retrieve_relevant_speeches_with_matches(data, tfidf_matrix, query, top_k=5):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = np.argsort(similarity)[-top_k:][::-1]
    retrieved_df = data.iloc[top_indices].copy()
    
    # Get matched words
    matched_words_list = []
    query_features = query_vec.nonzero()[1]
    query_terms = set([vectorizer.get_feature_names_out()[i] for i in query_features])

    for idx in top_indices:
        speech_vec = tfidf_matrix[idx]
        speech_features = speech_vec.nonzero()[1]
        speech_terms = set([vectorizer.get_feature_names_out()[i] for i in speech_features])
        common_terms = query_terms.intersection(speech_terms)
        matched_words_list.append(list(common_terms))

    retrieved_df["matched_words"] = matched_words_list
    retrieved_df["similarity"] = similarity[top_indices]
    return retrieved_df

In [11]:
def build_context(speeches_df, max_chars=1000):
    context = ""
    for _, row in speeches_df.iterrows():
        context += f"\n---\nSpeech by {row['president']}:\n{row['transcript'][:max_chars]}...\n"
    return context

## Asking an LLM, with and without RAG

In [19]:
host='127.0.0.1:65384'
model="gpt-oss:20b"

client = Client(
    host=host,
)

In [20]:
answer = client.chat(model, messages=[
        {
            'role': 'user',
            'content': f"""What did president Grover Cleveland say about immigration?"""
        },
    ])

print(answer.message.content)

### 1.  Cleveland’s General Philosophy

Grover Cleveland’s public statements show that he saw immigration as a **policy problem** rather than a moral crusade.  He believed that the United States had to:

*  **Regulate** the flow of people so that the nation could maintain its economic health and social stability.  
*  **Allow** those who were “fit” and “productive” while **excluding** those who were likely to become a burden on the public system.  

He summed this up in a famous remark that appears in several of his speeches and letters:

> **“The law is the only instrument we have for regulating immigration.”**  
>  —*Grover A. Cleveland, 1891, “Address to the Board of Immigration” (Washington, D.C.)*  

This line underlines Cleveland’s belief that a clear, enforceable legal framework was the only way to balance the benefits of immigration with the risks to the nation’s welfare.

### 2.  Concrete Statements

| Date & Source | Quote | What it Reveals |
|---------------|-------|--------

In [21]:
def query_ollama(context, question, client):
    messages = [
        {
            'role': 'user',
            'content': f"""Answer the following question based on the context:\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
        },
    ]

    for part in client.chat(model, messages=messages, stream=True):
        print(part.message.content, end='', flush=True)

In [22]:
query = "What did president Grover Cleveland say about immigration?"

# Retrieve
retrieved, scores = retrieve_relevant_speeches(data1, tfidf_matrix, query)
context = build_context(retrieved)

In [23]:
print(context)


---
Speech by Donald Trump:
THE PRESIDENT:Just a short time ago, I had the honor of presiding over the swearing-in of five new great American citizens.It was a beautiful ceremony and a moving reminder of our nations proud history of welcoming legal immigrants from all over the world into our national family.
 I told them that the beauty and majesty of citizenship is that it draws no distinctions of race, or class, or faith, or gender or background.All Americans, whether first generation or tenth generation, are bound together in love and loyalty, friendship and affection.
 We are all equal.We are one team, and one people, proudly saluting one great American flag.We believe in a safe and lawful system of immigration, one that upholds our laws, our traditions, and our most cherished values.
 Unfortunately, our immigration system has been badly broken for a very long time.Over the decades, many Presidents and many lawmakers have come and gone, and no real progress has been made on immigr

In [24]:
# Generate answer
query_ollama(context, query, client)

Grover Cleveland’s remarks were focused on a specific amendment to the 1891 immigration act.  
He explained that the bill he was returning would add a new category of excluded aliens:  

* **Persons who are physically capable and over 16 years old but cannot read or write English (or any other language).**  
* **Exceptions:**  
  * An older parent or grandparent (over 50 years old) who is the parent or grandparent of a qualified immigrant over 21 years old and who is able to support that immigrant may accompany or be brought to the United States.  
  * Likewise, a wife or minor child who cannot read or write may accompany such an adult, or such an adult may be sent for and join the family of a child or grandchild over 21 who is qualified and capable.

In short, Cleveland said the proposed amendment would broaden the list of excluded immigrants to those over 16 who are illiterate in English, but would carve out a special provision for older family members who can support a qualified adu

Grover Cleveland’s remarks were focused on a specific amendment to the 1891 immigration act.  
He explained that the bill he was returning would add a new category of excluded aliens:  

* **Persons who are physically capable and over 16 years old but cannot read or write English (or any other language).**  
* **Exceptions:**  
  * An older parent or grandparent (over 50 years old) who is the parent or grandparent of a qualified immigrant over 21 years old and who is able to support that immigrant may accompany or be brought to the United States.  
  * Likewise, a wife or minor child who cannot read or write may accompany such an adult, or such an adult may be sent for and join the family of a child or grandchild over 21 who is qualified and capable.

In short, Cleveland said the proposed amendment would broaden the list of excluded immigrants to those over 16 who are illiterate in English, but would carve out a special provision for older family members who can support a qualified adult immigrant.

### See what a context looks like

In [25]:
query = "What did the presidents say about immigration?"
results_df = retrieve_relevant_speeches_with_matches(data1, tfidf_matrix, query, top_k=10)

for _, row in results_df.iterrows():
    print(f"\n➡️ {row['president']}:")
    print(f"Similarity: {row['similarity']:.4f}")
    print(f"Matched words: {', '.join(row['matched_words'])}")
    print(f"Excerpt: {row['transcript'][:1500]}...\n")


➡️ Donald Trump:
Similarity: 0.5530
Matched words: immigration
Excerpt: THE PRESIDENT:Just a short time ago, I had the honor of presiding over the swearing-in of five new great American citizens.It was a beautiful ceremony and a moving reminder of our nations proud history of welcoming legal immigrants from all over the world into our national family.
 I told them that the beauty and majesty of citizenship is that it draws no distinctions of race, or class, or faith, or gender or background.All Americans, whether first generation or tenth generation, are bound together in love and loyalty, friendship and affection.
 We are all equal.We are one team, and one people, proudly saluting one great American flag.We believe in a safe and lawful system of immigration, one that upholds our laws, our traditions, and our most cherished values.
 Unfortunately, our immigration system has been badly broken for a very long time.Over the decades, many Presidents and many lawmakers have come and gone, 

In [27]:
query = "What did the presidents say about oil?"
results_df = retrieve_relevant_speeches_with_matches(data1, tfidf_matrix, query)

for _, row in results_df.iterrows():
    print(f"\n➡️ {row['president']}:")
    print(f"Similarity: {row['similarity']:.4f}")
    print(f"Matched words: {', '.join(row['matched_words'])}")
    print(f"Excerpt: {row['transcript'][:1500]}...\n")


➡️ Barack Obama:
Similarity: 0.6592
Matched words: oil
Excerpt: Good evening. As we speak, our nation faces a multitude of challenges. At home, our top priority is to recover and rebuild from a recession that has touched the lives of nearly every American. Abroad, our brave men and women in uniform are taking the fight to al Qaeda wherever it exists. And tonight, Ive returned from a trip to the Gulf Coast to speak with you about the battle were waging against an oil spill that is assaulting our shores and our citizens.
 On April 20th, an explosion ripped through BP Deepwater Horizon drilling rig, about 40 miles off the coast of Louisiana. Eleven workers lost their lives. Seventeen others were injured. And soon, nearly a mile beneath the surface of the ocean, oil began spewing into the water.
 Because there has never been a leak this size at this depth, stopping it has tested the limits of human technology. Thats why just after the rig sank, I assembled a team of our nations best scien

In [28]:
query = "What did the presidents say about oil?"

# Retrieve
retrieved, scores = retrieve_relevant_speeches(data1, tfidf_matrix, query)
context = build_context(retrieved)

# Generate answer
query_ollama(context, query, client)

**What the presidents said about oil**

| President | Main points about oil (or energy) |
|-----------|-----------------------------------|
| **Barack Obama** | • The Deepwater Horizon rig exploded and released an unprecedented oil spill off Louisiana. <br>• “We’re waging a battle against an oil spill that is assaulting our shores and our citizens.” <br>• Stopping the leak “has tested the limits of human technology,” so he assembled a national team of scientists and engineers to contain it. |
| **Jimmy Carter** | • In his first address, he framed the energy problem as “the greatest challenge” the nation would face in its lifetime, emphasizing that we must “balance our demand for energy with our rapidly shrinking resources.” <br>• In his follow‑up speech he announced the creation of a new Department of Energy and the “National Energy Plan,” underscoring the need for comprehensive federal action to address the looming crisis. |
| **Gerald Ford** | • He pledged a program to make the Unite

**What the presidents said about oil**

| President | Main points about oil (or energy) |
|-----------|-----------------------------------|
| **Barack Obama** | • The Deepwater Horizon rig exploded and released an unprecedented oil spill off Louisiana. <br>• “We’re waging a battle against an oil spill that is assaulting our shores and our citizens.” <br>• Stopping the leak “has tested the limits of human technology,” so he assembled a national team of scientists and engineers to contain it. |
| **Jimmy Carter** | • In his first address, he framed the energy problem as “the greatest challenge” the nation would face in its lifetime, emphasizing that we must “balance our demand for energy with our rapidly shrinking resources.” <br>• In his follow‑up speech he announced the creation of a new Department of Energy and the “National Energy Plan,” underscoring the need for comprehensive federal action to address the looming crisis. |
| **Gerald Ford** | • He pledged a program to make the United States “independent of foreign sources of energy by 1985.” <br>• He quoted that the country was dependent on about **37 %** of its petroleum needs from abroad and warned that in ten years it could import more than **half** of its oil. <br>• He highlighted the rising cost of foreign oil – from **$3 B** five years ago to **$25 B** per year now – and warned that the trend could continue if nothing is done. |

In short:  
- **Obama** spoke of the emergency response to an oil spill.  
- **Carter** warned of an impending energy crisis and called for new federal leadership and policy.  
- **Ford** focused on the United States’ dependence on imported oil and the need for energy independence.


In [29]:
query = "What did the presidents say about immigration from mexico?"

# Retrieve
retrieved, scores = retrieve_relevant_speeches(data1, tfidf_matrix, query)
context = build_context(retrieved)

# Generate answer
query_ollama(context, query, client)

The three speeches that were supplied contain no remarks that directly address immigration coming from Mexico.  

- **Donald Trump** speaks about a “safe and lawful” immigration system and the need to reform it, but he does not single out Mexico or Mexican immigrants.  
- **James K. Polk** talks about national prosperity, territorial expansion, and the influx of people in general; he never mentions Mexican immigration.  
- **John Tyler** discusses the Mexican government’s hostile language about the war with Texas and the pending annexation of Texas, but he does not discuss immigration from Mexico.  

So, in the excerpts provided, none of the presidents made a statement about immigration from Mexico.

The three speeches that were supplied contain no remarks that directly address immigration coming from Mexico.  

- **Donald Trump** speaks about a “safe and lawful” immigration system and the need to reform it, but he does not single out Mexico or Mexican immigrants.  
- **James K. Polk** talks about national prosperity, territorial expansion, and the influx of people in general; he never mentions Mexican immigration.  
- **John Tyler** discusses the Mexican government’s hostile language about the war with Texas and the pending annexation of Texas, but he does not discuss immigration from Mexico.  

So, in the excerpts provided, none of the presidents made a statement about immigration from Mexico.

In [30]:
query = "What did the presidents say about immigration from mexico?"
results_df = retrieve_relevant_speeches_with_matches(data1, tfidf_matrix, query)

for _, row in results_df.iterrows():
    print(f"\n➡️ {row['president']}:")
    print(f"Similarity: {row['similarity']:.4f}")
    print(f"Matched words: {', '.join(row['matched_words'])}")
    print(f"Excerpt: {row['transcript'][:1500]}...\n")


➡️ Donald Trump:
Similarity: 0.4273
Matched words: immigration
Excerpt: THE PRESIDENT:Just a short time ago, I had the honor of presiding over the swearing-in of five new great American citizens.It was a beautiful ceremony and a moving reminder of our nations proud history of welcoming legal immigrants from all over the world into our national family.
 I told them that the beauty and majesty of citizenship is that it draws no distinctions of race, or class, or faith, or gender or background.All Americans, whether first generation or tenth generation, are bound together in love and loyalty, friendship and affection.
 We are all equal.We are one team, and one people, proudly saluting one great American flag.We believe in a safe and lawful system of immigration, one that upholds our laws, our traditions, and our most cherished values.
 Unfortunately, our immigration system has been badly broken for a very long time.Over the decades, many Presidents and many lawmakers have come and gone, 

In [31]:
query = "What did the presidents say about freedom of speech?"

# Retrieve
retrieved, scores = retrieve_relevant_speeches(data1, tfidf_matrix, query)
context = build_context(retrieved)

# Generate answer
query_ollama(context, query, client)

None of the excerpts above contain any remarks by the presidents on freedom of speech. The speeches focus on issues such as state admission, civil‑rights enforcement, foreign‑policy victories, Watergate, and the rule of law, but they do not mention or discuss the right of free expression.

In [32]:
print(context)


---
Speech by Andrew Johnson:
To the House of Representatives:
I return without my signature a bill entitled "An act to admit the State of Arkansas to representation in Congress."
The approval of this bill would be an admission on the part of the Executive that the "Act for the more efficient government of the rebel States," passed March 2, 1867, and the acts supplementary thereto were proper and constitutional. My opinion, however, in reference to those measures has undergone no change, but, on the contrary, has been strengthened by the results which have attended their execution. Even were this not the case, I could not consent to a bill which is based upon the assumption either that by an act of rebellion of a portion of its people the State of Arkansas seceded from the Union, or that Congress may at its pleasure expel or exclude a State from the Union, or interrupt its relations with the Government by arbitrarily depriving it of representation in the Senate and House of Representa

# Chunking and Embeddings

In [34]:
def chunk_speeches_by_paragraph(df):
    chunks = []
    metadata = []
    for i, row in df.iterrows():
        paragraphs = [p.strip() for p in row['transcript'].split('\n') if len(p.strip()) > 50]
        for para in paragraphs:
            chunks.append(para)
            metadata.append({
                'president': row['president'],
                'speech_id': i
            })
    return chunks, metadata



In [33]:
chunks, meta = chunk_speeches_by_paragraph(data1)

In [35]:
chunks[:5]

['During the wars which for some time have unhappily prevailed among the powers of Europe the United States of America, firm in their principles of peace, have endeavored, by justice, by a regular discharge of all their national and social duties, and by every friendly office their situation has admitted, to maintain with all the belligerents their accustomed relations of friendship, hospitality, and commercial intercourse. Taking no part in the questions which animate these powers against each other, nor permitting themselves to entertain a wish but for the restoration of general peace, they have observed with good faith the neutrality they assumed, and they believe that no instance of a departure from its duties can be justly imputed to them by any nation. A free use of their harbors and waters, the means of refitting and of refreshment, of succor to their sick and suffering, have at all times and on equal principles been extended to all, and this, too, amidst a constant recurrence o

In [36]:
len(chunks)

53843

In [37]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # Small and fast

In [38]:
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1683 [00:00<?, ?it/s]

In [39]:
def retrieve_with_embeddings(query, chunk_embeddings, top_k=5):
    query_embedding = embedder.encode([query], convert_to_tensor=True)
    sims = cosine_similarity(query_embedding.cpu(), chunk_embeddings.cpu())[0]
    top_indices = np.argsort(sims)[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "transcript": chunks[idx],
            "similarity": float(sims[idx]),
            "president": meta[idx]['president'],
            "speech_id": meta[idx]['speech_id']
        })
    return results

In [40]:
query = "How did presidents talk about freedom of speech?"
results = retrieve_with_embeddings(query, chunk_embeddings)

for res in results:
    print(f"\n➡️ {res['president']} (Similarity: {res['similarity']:.4f})")
    print(res['transcript'])


➡️ Lyndon B. Johnson (Similarity: 0.6332)
Yet we know that he did not speak only for America. We know that the four freedoms are not secure in America when they are violently denied elsewhere in the world. We know, too, that it requires more than speeches to resist the international enemies of freedom. We know that men respond to deeds when they are deaf to words. Even the precious word "freedom" may become empty to those without the means to use it.

➡️ George W. Bush (Similarity: 0.5646)
You know, Presidents can try to avoid hard decisions and therefore avoid controversy. That's just not my nature. I'm the kind of person that, you know, is willing to take on hard tasks, and in times of war people get emotional; I understand that. Never really, you know, spent that much time, frankly, worrying about the loud voices. I of course hear them, but they didn't affect my policy, nor did they affect -- affect how I made decisions.

➡️ Lyndon B. Johnson (Similarity: 0.5582)
We have freedom of

In [41]:
def build_chunck_context(speeches_df):
    context = ""
    for row in speeches_df:
        context += f"\n---\nPart of speech by {row['president']}:\n{row['transcript']}\n"
    return context

In [42]:
query = "How did presidents talk about freedom of speech?"

# Retrieve
results = retrieve_with_embeddings(query, chunk_embeddings, top_k=10)
context = build_chunck_context(results)

# Generate answer
query_ollama(context, query, client)

**In short:** The presidents used “freedom of speech” as a moral touch‑stone, a shield against tyranny, and a rallying cry for action. They spoke of it as a core American value, warned that it can be eroded when politics or war dominate, and urged both the public and government to guard it from censorship, silence, and intimidation.

| President | What they emphasized | How it shows up in the speech |
|-----------|----------------------|--------------------------------|
| **Lyndon B. Johnson** | Freedom of speech is a *living* safeguard that must be defended and exercised without restraint. | “We have freedom of speech… we have freedom of assembly… I have seen no restraints imposed by anybody.” |
| **George W. Bush** | Speech is a tool of decision‑making; politicians should not be swayed by loud voices but must confront hard realities. | “Presidents can try to avoid hard decisions… I am the kind of person that is willing to take on hard tasks.” |
| **Donald Trump** | Freedom of speech 

**In short:** The presidents used “freedom of speech” as a moral touch‑stone, a shield against tyranny, and a rallying cry for action. They spoke of it as a core American value, warned that it can be eroded when politics or war dominate, and urged both the public and government to guard it from censorship, silence, and intimidation.

| President | What they emphasized | How it shows up in the speech |
|-----------|----------------------|--------------------------------|
| **Lyndon B. Johnson** | Freedom of speech is a *living* safeguard that must be defended and exercised without restraint. | “We have freedom of speech… we have freedom of assembly… I have seen no restraints imposed by anybody.” |
| **George W. Bush** | Speech is a tool of decision‑making; politicians should not be swayed by loud voices but must confront hard realities. | “Presidents can try to avoid hard decisions… I am the kind of person that is willing to take on hard tasks.” |
| **Donald Trump** | Freedom of speech is under threat from cancel‑culture and censorship; it is essential to keep the nation united. | “Unprecedented assault on free speech… the efforts to censor, cancel, and blacklist… are wrong and dangerous.” |
| **Franklin D. Roosevelt** | Speech is a universal right that must be protected worldwide. | “The first is freedom of speech and expression everywhere in the world.” |
| **Richard M. Nixon** | Confidentiality is necessary, but open discussion is vital to resolve conflict. | “The principle of confidentiality… but the same brutal candor is necessary in discussing how to bring warring factions to the peace table.” |
| **Joe Biden** | Freedom of speech is part of the American ideal and cannot be ignored when rights are violated. | “No responsible American President could remain silent when basic human rights are being so blatantly violated.” |

**Key themes**

1. **Defining freedom of speech as a cornerstone of democracy.**  
2. **Highlighting its fragility**—when other powers (war, politics, censorship) press against it.  
3. **Calling for action**—not just speeches but deeds that protect and expand the right.  
4. **Warning against censorship** and cancel culture.  
5. **Framing it as a moral obligation** for both presidents and citizens.  

These speeches collectively illustrate that U.S. presidents consistently framed freedom of speech as a protective, unifying, and indispensable value that must be defended against every threat, whether domestic or international.


In [43]:
query = "Which president talked about immigration and immigrants positively in their speech?"

# Retrieve
results = retrieve_with_embeddings(query, chunk_embeddings, top_k=10)
context = build_chunck_context(results)

# Generate answer
query_ollama(context, query, client)

Donald Trump.

In [44]:
print(context)


---
Part of speech by Donald Trump:
THE PRESIDENT:Just a short time ago, I had the honor of presiding over the swearing-in of five new great American citizens.It was a beautiful ceremony and a moving reminder of our nations proud history of welcoming legal immigrants from all over the world into our national family.

---
Part of speech by Joe Biden:
THE PRESIDENT: First of all, all the policies that were underway were not helping at all did not slow up the amount of immigration and theres many people coming.

---
Part of speech by Joe Biden:
THE PRESIDENT: No, no, but I mean I dont know how much detail you want about immigration. Maybe Ill stop there and fin-

---
Part of speech by Donald Trump:
THE PRESIDENT: I had a good talk with the governor of Georgia.

---
Part of speech by Lyndon B. Johnson:
THE PRESIDENT. The Governor had his share of the conversation. He told me of the problems that he had in Alabama, the fears that he entertained, and he expressed the hope that I could do so

# *To Be Continued..*